In [1]:
"""Code for Exercise 1 on Exercise sheet 6. The goal of this Exercise is to check whether the PKCS#7 padding is 
correct or not."""

BLOCK_SIZE = 16


def main():
    """Check the correctness of the padding checking."""
    good_padding_1 = b"Hello world\x05\x05\x05\x05\x05"
    good_padding_2 = b"Fancy cryptology" + b"\x10" * 16
    assert is_padding_good(good_padding_1)
    assert is_padding_good(good_padding_2)
    bad_padding_1 = b"Hello world\x04\x04\x04"
    bad_padding_2 = b"Hi world\x04\x04\x04\x04"
    assert not is_padding_good(bad_padding_1)
    assert not is_padding_good(bad_padding_2)


def is_padding_good(padded_message: bytes) -> bool:
    """Verify that the message has been padded with a valid padding."""

    # YOUR CODE SHOULD GO HERE
    if len(padded_message) % 16 != 0:
        return False
    else:
        i = padded_message[-1]
        for j in range(1,i+1):
            if (padded_message[-j] != i):
                return False
    return True


if __name__ == "__main__":
    main()
    

In [2]:
"""Code for Exercise 3 on Exercise sheet 6."""
import base64

# You need the `pycryptodome` package for these modules
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

BLOCK_SIZE = 16


def main():
    """Pass a ciphertext together with the IV and an oracle to the attacker."""
    ciphertext = create_ciphertext()
    attack(ciphertext, IV, padding_oracle)
    
    
def attack(ciphertext: bytes, iv: bytes, oracle):
    """Determine the plaintext from the ciphertext using the oracle."""
    
    number_of_blocks = int(len(ciphertext) / BLOCK_SIZE)
    plaintext = [0] * BLOCK_SIZE * number_of_blocks
   
    for block in range(number_of_blocks):
        # for the first block, iv is the iv that was given. For the following blocks, the iv is the ciphertext,
        # from the previous block, i.e. iv for block 2 is ciphertext from block 1
        newIV = iv
        current_block = ciphertext[block*16:(block+1)*16]
    
        # first step is to determine last byte of current block
        for i in range(255):
            newIV = ith_IV_for_current_byte(iv, newIV, 0, i)
            cipher = AES.new(KEY, AES.MODE_CBC, newIV)
            # if the padding is correct, we can continue to second last byte of the current block
            if (oracle(current_block, newIV) == True and cipher.decrypt(current_block)[-1] == 1):
                plaintext[15 + block * BLOCK_SIZE] = i ^ 1
                break
  
        # next step is to determine second last byte of current block, then third last byte, ect.
        for j in range(1,BLOCK_SIZE):
            newIV = IV_next_byte_in_current_block(iv, newIV, j)
 
            for i in range(255):
                newIV = ith_IV_for_current_byte(iv, newIV, j, i)
                # if the padding is correct, we can continue to the next byte (which means the byte before the
                # current, since we determine the bytes in the order 16, 15, ...)
                if (oracle(current_block, newIV) == True):
                    plaintext[BLOCK_SIZE - j - 1 + block * BLOCK_SIZE] = i ^ (j + 1)
                    break

        # update the iv to the current (previous) block
        iv = current_block

    print("".join(map(chr, plaintext)))


def IV_next_byte_in_current_block(iv: bytes, current_newIV: bytes, current_reverse_byte: int) -> bytes:
    """Update IV such that it can be used to determine the next bytes. For example: When the last byte of the
    current block has been determined (by manipulating with the IV such that the last byte of the decrypted
    message becomes 0x01), then this function is used to force the last (and already determined) byte of the
    decrypted message to equal 0x02, ect."""
    
    IV = iv[0:BLOCK_SIZE-current_reverse_byte]
    for i in range(current_reverse_byte):
        IV = IV + bytes([current_newIV[-(current_reverse_byte-i)] ^ IV_vector[current_reverse_byte-1]])
    
    return IV
    
def ith_IV_for_current_byte(iv: bytes, current_newIV: bytes, current_reverse_byte: int, current_i: int) -> bytes:
    """Update IV for the current byte. This function is the one that manipulates with the IV by keep adding values
    to the current byte, until the padding oracle verifies the correctness of the padding."""
    
    IV = current_newIV[:(15-current_reverse_byte)] + bytes([iv[-(current_reverse_byte+1)] ^ current_i])
    for j in range(current_reverse_byte):
        IV = IV + bytes([current_newIV[-(current_reverse_byte-j)]])
    
    return IV
    
    
# the following IV vector contains the numbers that we should XOR with in order to get the next number, in the range
# 1 to 16. I.e. in order to get 2 from 1, we XOR 1 with 0x03. In order to get 3 from 2, we XOR 2 with 0x01, ect.
IV_vector = [0x03,0x01,0x07,0x01,0x03,0x01,0xF,0x01,0x03,0x01,0x07,0x01,0x03,0x01,0x1F]

KEY = get_random_bytes(16)
IV = get_random_bytes(16)
SECRET = (
    b"SXQncyBiZWVuIGEgaGFyZCBkYXkncyBuaWdodCwgYW5kIEkndmUgYmVlbiB3b3Jr"
    b"aW5nIGxpa2UgYSBkb2c="
)


def create_ciphertext() -> bytes:
    """Return the secret after padding and encrypting in CBC mode."""
    cipher = AES.new(KEY, AES.MODE_CBC, IV)
    secret = base64.b64decode(SECRET)
    padded_secret = pkcs7_pad(secret, BLOCK_SIZE)
    ciphertext = cipher.encrypt(padded_secret)
    return ciphertext


def padding_oracle(ciphertext: bytes, initialization_vector: bytes) -> bool:
    """Verify that the ciphertext contains correct padding after decryption."""
    cipher = AES.new(KEY, AES.MODE_CBC, initialization_vector)
    plaintext = cipher.decrypt(ciphertext)
    return is_padding_good(plaintext)


def is_padding_good(padded_message: bytes) -> bool:
    """Verify that the padding of the message is valid."""
    if not padded_message or len(padded_message) % BLOCK_SIZE != 0:
        return False
    last_byte = padded_message[-1]
    padding_length = int(last_byte)
    if padding_length > len(padded_message):
        return False
    if not all(byte == last_byte for byte in padded_message[-padding_length:]):
        return False
    return True


def pkcs7_pad(message: bytes, bytes_per_block: int) -> bytes:
    """Return the message padded to a multiple of `bytes_per_block`."""
    if bytes_per_block >= 256 or bytes_per_block < 1:
        raise Exception("Invalid padding modulus")
    remainder = len(message) % bytes_per_block
    padding_length = bytes_per_block - remainder
    padding = bytes([padding_length] * padding_length)
    return message + padding


if __name__ == "__main__":
    main()


It's been a hard day's night, and I've been working like a dog
